# Индивидуальное задание 1

**Выполнил:** Жаров Игорь Андреевич

**Группа:** 1

---

## Задание:
классифицировать вино по его химическим свойствам. Важное уточнение: хоть это всё ещё и является предсказанием от нейросети, сама задача считается задачей классификации.

В столбце 'target' находятся значения принадлежности к классу: 0, 1, 2.
- Класс 0: Вина первого сорта.
- Класс 1: Вина второго сорта.
- Класс 2: Вина третьего сорта.

Сгружаем датасет, смотрим его, разделяем на тестовую и тренировочную выборки, нормализуем данные, создаем архитектуру нейронной сети и обучаем. А после на тестовых данных проверяем результат нашей работы.

In [14]:
!pip install numpy pandas scikit-learn tensorflow

Импортируем необходимые библиотеки

In [15]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
import tensorflow as tf

# Споcоб 1

## Работа с данными

**One-hot encoding** — это способ представления категориальных данных. Для каждого класса создаётся вектор, который имеет единицу на месте, соответствующем данному классу, и нули на всех остальных позициях.

Пример для трёх классов:

- Класс 0: [1, 0, 0]
- Класс 1: [0, 1, 0]
- Класс 2: [0, 0, 1]

Такой способ представления данных лучше подходит для задач классификации.

**Зачем это нужно?** В задачах многоклассовой классификации нейронные сети на выходе имеют несколько нейронов (по числу классов), и для корректного обучения нужно, чтобы целевые метки были представлены в виде вектора. Каждый класс представляется вектором, где одна позиция имеет значение 1, а остальные — 0. Этот формат называется one-hot encoding.

**Далее выгружаем датасет**, разделяем данные на признаки и метки классов. Используем ont-hot encoding и объединяем все данные в один датафрейм для удобного просмотра.

In [29]:
load_data = load_wine()
data = load_data['data']
target = load_data['target']
target = tf.keras.utils.to_categorical(target, 3) # используем one-hot encoding
# Выводим данные до нормализации (убеждаемся, что данные импортированы)
# print(pd.DataFrame(X).head())
# print(pd.DataFrame(y).head())
dataframe = pd.DataFrame(data, columns=load_data['feature_names'])
dataframe['target'] = pd.Categorical(load_data['target'])
dataframe.head(7)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
5,14.20,1.76,2.45,15.2,112.0,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450.0,0
6,14.39,1.87,2.45,14.6,96.0,2.50,2.52,0.30,1.98,5.25,1.02,3.58,1290.0,0


## Разделение данных на тренировочные и тестовые

Далее разделяем данные на тренировочные и тестовые

**train_test_split**: Этот метод из библиотеки scikit-learn разделяет данные на тренировочную и тестовую выборки.

X — это ваши признаки (химические свойства вина).

y — это целевые метки (классы вин: 0, 1, 2).

test_size=0.2: 20% данных отводятся под тестирование, а 80% — для обучения.

random_state=42: Это фиксирует "случайность" в разбиении данных, чтобы результаты были воспроизводимыми при каждом запуске.

В результате этого шага данные делятся на тренировочные и тестовые:

X_train и y_train — тренировочные данные для обучения модели.

X_test и y_test — тестовые данные для проверки модели.

In [17]:
from sklearn.model_selection import train_test_split
train_size = int(0.8 * dataframe.shape[0])
#X_train, X_test = data[:train_size], data[train_size:]
#y_train, y_test = target[:train_size], target[train_size:]
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)
# dataframe_train = pd.DataFrame(X_train, columns=load_data['feature_names'])
# dataframe_train.head(7)

## Нормализация данных

**StandardScaler**: Это один из методов нормализации данных, который приводится к стандартному нормальному распределению (среднее значение 0 и стандартное отклонение 1).

**fit_transform(X_train)**:

fit: Этот метод вычисляет параметры нормализации (среднее значение и стандартное отклонение) на тренировочных данных.

transform: Применяет вычисленные параметры для преобразования данных. Для каждого признака выполняется следующее преобразование:

In [18]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
'''
print(f"\nЧисло признаков до нормализации: {X_train.shape[1]}")
print(f"Число признаков после нормализации: {X_train_scaled.shape[1]}")
print(f"Число имен признаков: {len(load_data.feature_names)}")'''
df_train_scaled = pd.DataFrame(X_train_scaled, columns=load_data['feature_names'])
df_train_scaled.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1.665293,-0.608406,1.218962,1.605400,-0.167384,0.804002,-0.691678,1.267226,1.877540,3.419473,-1.656329,-0.879409,-0.248606
1,-0.549525,2.751541,1.003315,1.605400,-0.304379,-0.785384,-1.401233,2.049600,-0.873505,-0.024801,-0.584633,-1.254621,-0.729922
2,-0.745310,-1.143541,-0.937507,-0.282704,-0.852357,1.937029,1.746791,-1.001659,0.587987,-0.240068,0.358460,0.246227,-0.248606
3,0.612948,-0.617179,1.003315,0.879206,-0.783860,0.489272,-0.901547,1.188988,1.172585,2.881305,-1.656329,-1.129550,-0.381383
4,0.111249,-0.766315,-0.937507,-1.154137,-0.167384,0.174542,0.637487,-0.688710,-0.409266,-0.584496,0.958609,0.135053,0.946386


## Работа с моделью

Инициализируем саму модель. Создаем 2 закрытых слоя по 32 нейрона. Для такой простой модели больше ставить не имеет смысла. А также присутствуют 3 выходных нейрона для получения результатов.

Если задать большее количетство нейронов, то модель будет делать более точные предсказания, за счет увелечения параметров. Модель после обучения почти не будет допускать ошибок.

In [30]:
from tensorflow.keras import layers, models
model = models.Sequential()
model.add(layers.InputLayer(input_shape=(X_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                     │ (None, 64)                  │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,075 (12.01 KB)

 Trainable params: 3,075 (12.01 KB)

 Non-trainable params: 0 (0.00 B)

### Компиляция модели

Далее коомпилируем модель. При обучении модели можно задать количество эпох для обучения и размер пакета, определяет количество обучающих примеров, используемых для оценки градиента и обновления весов модели за одну итерацию.

Меняя эти настройки можно получить совершенно разный результат работы после обучения модели. Например, если выставить в качестве количества эпох значение 100, а размер пакетов 16. То, модель почти перестанет делать ошибки в предсказаниях, а процентаж для каждой итерации вырастет до идеальных значений.

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
# 5. Обучение модели
history = model.fit(X_train_scaled, y_train, epochs=25, batch_size=8, validation_split=0.2)
counter = 10 # Количество выводимых записей

# 6. Оценка на тестовых данных
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"\nAccuracy: {test_acc}")

# Прогнозы на тестовых данных
y_probs = model.predict(X_test_scaled)
y_pred = np.argmax(y_probs, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

df_predictions = pd.DataFrame({
    'Правильный сорт': y_test_labels,
    'Предсказанный сорт': y_pred
})
print(df_predictions.head(counter))
print("\nВероятности для каждого примера:")
for i, probs in enumerate(y_probs[:counter]):
    print(f"Пример {i}: Вероятности - Класс 0: {probs[0]:.4f}, Класс 1: {probs[1]:.4f}, Класс 2: {probs[2]:.4f}")

Epoch 1/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.2329 - loss: 1.2108 - val_accuracy: 0.5172 - val_loss: 0.9002
Epoch 2/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7799 - loss: 0.8315 - val_accuracy: 0.8621 - val_loss: 0.6880
Epoch 3/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.9476 - loss: 0.5866 - val_accuracy: 0.8966 - val_loss: 0.5169
Epoch 4/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9354 - loss: 0.4671 - val_accuracy: 0.9310 - val_loss: 0.3752
Epoch 5/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9703 - loss: 0.3162 - val_accuracy: 0.9655 - val_loss: 0.2735
Epoch 6/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9744 - loss: 0.2493 - val_accuracy: 1.0000 - val_loss: 0.2015
Epoch 7/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9476 - loss: 0.1794 - val_accuracy: 1.0000 - val_loss: 0.1574
Epoch 8/25
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9767 - loss: 0.1268 - val_accuracy: 1.0000 - va

---

# Тестовые блоки

Тестовая модель

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
data = load_wine()
X = data.data
y = data.target

# Преобразуем в DataFrame для удобства
df = pd.DataFrame(X, columns=data.feature_names)
df['target'] = y

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормализация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Создание модели
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1, validation_split=0.2)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Тестовая точность: {accuracy:.2f}')
predictions = model.predict(X_test)
for i in range(len(predictions)):
    print(f'Пример {i + 1}: Вероятности - Класс 0: {predictions[i][0]:.4f}, Класс 1: {predictions[i][1]:.4f}, Класс 2: {predictions[i][2]:.4f}')
predicted_classes = np.argmax(predictions, axis=1)
print("Предсказанные классы:", predicted_classes)
print("Реальные классы:", y_test)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.3172 - loss: 1.0764 - val_accuracy: 0.4483 - val_loss: 0.9899
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5771 - loss: 0.9310 - val_accuracy: 0.7241 - val_loss: 0.8833
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6834 - loss: 0.8540 - val_accuracy: 0.8276 - val_loss: 0.7948
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6972 - loss: 0.7788 - val_accuracy: 0.8966 - val_loss: 0.7201
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7792 - loss: 0.7136 - val_accuracy: 0.8966 - val_loss: 0.6573
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8286 - loss: 0.6362 - val_accuracy: 0.8966 - val_loss: 0.5972
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8547 - loss: 0.6065 - val_accuracy: 0.8966 - val_loss: 0.5373
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9237 - loss: 0.5327 - val_accuracy: 0.8966 - val_loss: 0.4796
Epoch 9/50
8/8 ━━